# Test Code for the Overlap with the RGES Footprint

Code adapted from an example by Lynne Jones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf

Load the baseline v2.0 as a test case OpSim:

In [2]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [3]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [4]:
import numpy as np
import healpy as hp
from astropy import units as u
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic, TETE, SkyCoord

class overlapRGESFootprintMetric(maf.BaseMetric):
    """Metric to evaluate the survey footprint overlap survey region for the Roman Galactic Exoplanet Survey

    Derived from the SpacialOverlapMetric by Michael Lund and revised and updated by Rachel Street
    
    Parameters
    ----------
    fieldRA : float, RA in degrees of a given pointing
    fieldDec : float, Dec in degrees of a given pointing
    filter : str, filter bandpass used for a given observation
    """
    
    def __init__(self, cols=['fieldRA','fieldDec','filter','fiveSigmaDepth'],
                       metricName='overlapRGESFootprintMetric',
                       **kwargs):
        """Kwargs must contain:
        filters  list Filterset over which to compute the metric
        """

        self.ra_col = 'fieldRA'
        self.dec_col = 'fieldDec'
        self.filterCol = 'filter'
        self.m5Col = 'fiveSigmaDepth'
        self.filters = ['u','g', 'r', 'i', 'z', 'y']
        self.magLimit = 22.0
        cwd = os.getcwd()
        self.create_map()
        self.load_RGES_footprint()
        
        super().__init__(col=cols, metricName=metricName)

    def create_map(self):
        self.NSIDE = 64
        self.ahp = HEALPix(nside=self.NSIDE, order='ring', frame=TETE())
        
    def load_RGES_footprint(self):
        
        # Location of the RGES survey field in Galactic Coordinates.  
        # This is defined as a single pointing, since the survey
        # region will be ~2sq deg and fully encompassed within a single Rubin pointing
        l_center = 2.216
        b_center = -3.14
        l_width = 3.5
        b_height = 3.5
        n_points = 50
                
        # Represent coordinate pointings within this region as a meshgrid
        halfwidth_l = l_width / 2.0
        halfheight_b = b_height / 2.0

        l_min = max( (l_center-halfwidth_l), 0 )
        l_max = min( (l_center+halfwidth_l), 360.0 )
        b_min = max( (b_center-halfheight_b), -90.0 )
        b_max = min( (b_center+halfheight_b), 90.0 )

        l = np.linspace(l_min, l_max, n_points) * u.deg
        b = np.linspace(b_min, b_max, n_points) * u.deg

        LL,BB = np.meshgrid(l, b)
        
        coords = SkyCoord(LL, BB, frame=Galactic())

        # Calculate the corresponding HEALpixels
        pixels = self.ahp.skycoord_to_healpix(coords)
        self.RGES_pixels = np.unique(pixels.flatten())

    def run(self, dataSlice, slicePoint=None):

        # Only count observations with adequate S/N
        match = np.where(dataSlice[self.m5Col] >= self.magLimit)[0]

        # Extract the RA,Dec coordinates of the fields surveyed by matching observations the dataSlice
        # and calculate which HEALpixels these correspond to
        coords_icrs = SkyCoord(dataSlice[self.ra_col][match], dataSlice[self.dec_col][match], frame='icrs', unit=(u.deg, u.deg))
        coords_gal = coords_icrs.transform_to(Galactic()) 

        surveyed_pixels = self.ahp.skycoord_to_healpix(coords_gal)
        surveyed_pixels = np.unique(surveyed_pixels.flatten())
        
        # Calculate the fraction of the RGES survey pixels included in the surveyed pixels
        overlapping_pixels = set(self.RGES_pixels).intersection(set(surveyed_pixels))
        
        metric_value = float(len(overlapping_pixels)) / float(len(self.RGES_pixels))
        
        return metric_value

In [5]:
mymetric = overlapRGESFootprintMetric()
sqlconstraint = None
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)
g.runAll()

Querying database None with no constraint for columns ['rotSkyPos', 'fieldDec', 'fieldRA', 'fiveSigmaDepth', 'filter'].
Found 2086980 visits
Running:  ['test_metric']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [6]:
bundle.metricValues

masked_array(data=[0.625],
             mask=[False],
       fill_value=-666.0)